In [1]:
from bokeh.models import ColumnDataSource, Whisker, Range1d
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool


from bokeh.io import output_notebook
output_notebook()

import numpy as np

Loading BokehJS ...

In [2]:
# M, R from
# 2007A%26A...468..353G: T Tau, BP Tau; teff from Spt in  electronic version of table 14 in https://iopscience.iop.org/article/10.1088/0004-637X/786/2/97
# https://ui.adsabs.harvard.edu/abs/2002A%26A...391..595S/abstract: RU Lup
# https://iopscience.iop.org/article/10.3847/1538-4357/aa8094: DF Tau
# V4046 Sgr: Donati 2011
# GM Aur: electronic version of table 14 in https://iopscience.iop.org/article/10.1088/0004-637X/786/2/97
# TW Hya: electronic version of table 14 in https://iopscience.iop.org/article/10.1088/0004-637X/786/2/97
# SU Aur: electronic version of table 14 in https://iopscience.iop.org/article/10.1088/0004-637X/786/2/97
# MP Mus: https://ui.adsabs.harvard.edu/abs/2007A%26A...465L...5A/abstract
#
# If the source only has a SpT, I convert to Teff with https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt


gs08tab = '''
nh,nh_err_d,nh_err_u,av,name, mass, radius, teff
1.8,-.3,+.2,0.07,RU Lup, 0.8, 1.7,3950
5,-0.6,+1,.3,T Tau, 2.41, 3.62,5270
6,-3,+2,.5,DF Tau,0.55+0.55 (binary),2.1+2.1,3560
2.2,-.63,+.13,0.01,V4046 Sgr,0.91+0.88 (binary),1.12+1.04, 4250
.3,-.1,+.1,.31,GM Aur, 0.9, 1.3, 4300
.35,-.05,+.04,0.01,TW Hya,0.69, 0.97, 3770
# .5,-.5,+.5,.26,AB Aur
# .76,-.19,+.19,.25,HD 163296
3.1,-.1,+.1,.51,SU Aur,2.07, 2.72, 5680
1.5,-.1,+.1,.5,BP Tau, 0.75, 1.97,3770
.46,-.45,.45,.17,MP Mus, 1.2, 1.3, 5170
# 13,-4,+1,0.78,AA Tau (variable)
# AA Tau +- 4, but the does not fit on plot
# V4046,Tw Hya Su Aur and Ab Aur are shifted in Av +0.01 for plotting clearity
# AB Aur and HD 163296 are originally part of the sample, I comment them out here for a comparative plot
'''
gs08tab = np.genfromtxt('../data/guenther_schmitt_2008.tab', delimiter=',', names=True,
              dtype=[float, float, float, float, 'U10', float, float, float])
gs08 = ColumnDataSource(data=dict(
    nh=gs08tab['nh'],
    nhlo=gs08tab['nh'] + gs08tab['nh_err_d'],
    nhhi=gs08tab['nh'] + gs08tab['nh_err_u'],
    av=gs08tab['av'],
    name=gs08tab['name'],
    mass=gs08tab['mass'],
    radius=gs08tab['radius'],
    teff=gs08tab['teff'],
))

In [3]:
TOOLTIPS = [
    ("object", "@name"),
    ("T_eff", "@teff K"),
    ("mass", "@mass{0.0} M☉"),
    ("radius", "@radius{0.0} R☉"),
]

TOOLTIPS_NO_NUMERIC = [
    ("object", "@name"),
    ("T_eff", "@teff K"),
    ("mass", "@mass M☉"),
    ("radius", "@radius R☉"),
]

fig_kwargs = {'width': 600, 'height': 400, 'tools': "pan,wheel_zoom,box_zoom,reset,save"}

In [4]:
aataunh = np.array([0.95, 1.54, 1.80, 1.36, 1.33, 1.04, 1.54, 1.02, 1.53, 0.6]) * 10
aatau = ColumnDataSource(data=dict(
    nh=aataunh,
    nhlo=aataunh - 4,
    nhhi=aataunh + 4,
    av=[0.78] * 9 + [4],
    name=['AA Tau'] * 10,
    teff=[4060] * 10, # 2007A%26A...468..353G
    lbol = [0.8] * 10,# 2007A%26A...468..353G
    logage = np.log10([2.4e6] * 10),# 2007A%26A...468..353G
    # In the NH/AV plot, I format the hover tool as string because of the binaries in the GS08 sample.
    # So, I can decide on the number of digits here by passing in a string.
    mass= ["0.76"] * 10,# 2007A%26A...468..353G
    radius=["1.81"] * 10,# 2007A%26A...468..353G
))

In [5]:
# data from https://ui.adsabs.harvard.edu/abs/2016MNRAS.459.2097P/abstract
twa30 = ColumnDataSource(data=dict(
    nh=[9.5, 1.6],
    nhlo=[9.5 - 6.1, 1.6 - 0.6],
    nhhi=[9.5 + 17.5, 1.6 + 1],
    av=[14.9, 4.7],
    name=['TWA 30A'] * 2,
    teff = [3050, 3050],
    logage = np.log10([8e6, 8e6]),
    lbol = np.array([7.6e31, 7.6e31]) / 3.8e33,
    radius = ["0.5", "0.5"],
    mass = ["0.1", "0.1"], # Principe et al does not give a mass, but says "will later evolve to M7."
                      # So, I'm putting the mass of an M7 MS star here.
))

In [6]:
p = figure(**fig_kwargs,
               y_axis_type="log",
               x_axis_type="log",
               )

hvr = HoverTool(tooltips=TOOLTIPS_NO_NUMERIC)
p.add_tools(hvr)
patches_to_hover = []

for source, color, label in [(gs08, 'red', 'GS08'),
                             (twa30, 'blue', 'TWA 30A'), #'Principe et al. (2016)'),
                             (aatau, 'orange', 'AA Tau'), #'Grosso et al. (2007)\nSchneider et al (2015)'),
                             #(aatau2, 'yellow', 'Schneider et al. (2007)'),
                             ]:
        for av, up, down in zip(source.data['av'],
                               source.data['nhhi'],
                               source.data['nhlo']):
            p.line([av, av], [up, down], color=color, legend_label=label,
                  line_width=2)

        circ = p.circle('av', 'nh', size=10, source=source, color=color,
                legend_label=label)
        patches_to_hover.append(circ)

p.xaxis.axis_label = r'$$A_V\; \mathrm{[mag]}$$'
p.yaxis.axis_label = r'$$N_H [10^{21}\; \mathrm{cm}^{-2}]$$'

x = np.array([0.01, 100])
p.line(x,  x  * 1.8, 
       line_color='green', line_width=2, 
       legend_label='ISM', muted=True)
p.line(x, x*0.63 * 1.12, 
       line_color='green', line_dash='dotted', line_width=2,
        legend_label='Serpens', muted=True)
p.line(x, x*1.4 * 1.12, 
       line_color='green', line_dash='dashed', line_width=2,
       legend_label=r'ρ Oph', muted=True)

# It would be nice to have a legend that allows to hide/show the data
# but it currently does not work for whiskers because they are not "glyphs"
# Could replace them with a set of line all with the same color (and same legend entry so they are grouped together)
# but I'm not sure it's worth the effort at this point.
# Get something done first, then we can improve it later.
p.legend.click_policy="mute"

p.legend.ncols=2
p.y_range = Range1d(1e-2, 1e3)

#// the figure's hovertool will by default, trigger for all elements
#// BUT we can modify this thanks to HoverTool having a renderers property, which takes a list of renderers

hvr.renderers = patches_to_hover

# activate to write out file, but also opens in new browser tab every time which is mildly annoying in development.
output_file('nhav.html', title='NH vs AV')

show(p)